<h1 style="font-family: Georgia; font-size:3em;color:#2462C0; font-style:bold">
Character-Level Language Model</h1><br>

Have you ever wondered how Gmail automatic reply works? Or how a Neural Network can generate musical notes? The general way of generating sequence of text is to train a model to predict the next word/character given all previous words/characters. Such model is called **Statistical Language Model**. So what is a statistical language model? A statistical language model tries to capture the statistical structure (latent space) of training text it's trained on. Usually **Recurrent Neural Network (RNN)** models family is used train the model due to the fact that it's very powerful and expressive in which they remember and process past information through their high dimensional hidden state units. The main goal of any language model is to learn the joint probability distribution of sequences of characters/words in a training text, i.e. trying to learn the joint probability function. For example, if we're trying to predict a sequence of $T$ words, we try to get the joint probability $P(w_1, w_2, ..., w_T)$ as big we we can which is equal to the product of all conditional probabilities $\prod_{t = 1}^T P(w_t/w_{t-1})$ at all time steps (t).

In this notebook, we'll cover **Character-level Language Model** where almost all the concepts hold for any other language models such as word-language models. The main task of character-level language model is to predict next character given all previous characters in a sequence of data, i.e. generate text character by character. More formally, given a training sequence $(x^1, ... , x^T)$, the RNN uses the sequence of its output vectors $(o^1, ... , o^T)$ to obtain a sequence of predictive distributions $P(x^t|x^{<t}) = softmax(o^t)$.

Let's illustrate how the character-level language model works using my first name ("imad") as an example (see figure 1 for all the details of this example).
1. We first build a vocabulary dictionary using all the unique letters of the names in the corpus as keys and the index of each letter starting from zero (since python is a zero-index language) in ascending order. For our example, the vocabulary dictionary would be: {"a": 0, "d": 1, "i": 2, "m": 3}. Therefore, "imad" would become a list of the following integers: [2, 3, 0, 1].
2. Convert the input and the output characters to lists of integers using the vocabulary dictionary. In this notebook, we'll assume that $x^1 = \vec{0}$ for all examples. Therefore, $y = "imad"$ and $x = \vec{0}\ + "ima"$. In other words, $x^{t + 1} = y^t$ which gives us: $y = [2, 3, 0, 1]$ and $x = [\vec{0}, 2, 3, 0]$.
3. For each character in the input:
    1. Convert the input characters into one-hot vectors. Notice how the first character $x^1 = \vec{0}$.
    2. Compute the hidden state layer.
    3. Compute the output layer and then pass it through softmax to get the results as probabilities.
    4. Feed the target character at time step (t) as the input character at time step $(t + 1)$.
    5. Go back to step A and repeat until we finish all the letters in the name.

The objective is to make the green numbers as big as we can and the red numbers as small as we can in the probability distribution layer. The reason for that is that the true index should have the highest probability by making it as close as we can to 1. The way to do that is to measure the loss using cross-entropy and the compute the gradients of the loss w.r.t. all parameters to update them in the opposite of the gradient direction. Repeating the process over many times where each time we adjust the parameters based on the gradient direction --> model will be able to correctly predict next characters given all previous one using all names in the training text. Notice that hidden state $h^4$ has all past information about all characters.

<p align="left">
<img src="posts_images/char_level_model/char_level_example.png"; style="width: 800px; height: 600px"><br>
<caption><center><u><b><font color="purple">Figure 1:</font></b></u> Illustrative example of character-level language model using RNN</center></caption>
</p>

<h2 style="font-family: Georgia; font-size:2em;color:purple; font-style:bold">
Training</h2>

The [dataset](http://deron.meranda.us/data/census-derived-all-first.txt) we'll be using has 5,163 names: 4,275 male names, 1,219 female names, and 331 names that can be both female and male names. The RNN architecture we'll be using to train the character-level language model is called **many to many** where time steps of the input $(T_x)$ = time steps of the output $(T_y)$. In other words, the sequence of the input and output are synced (see figure 2).
<p align="left">
<img src="posts_images/char_level_model/rnn_architecture.PNG"; style="width: 600px; height: 600px"><br>
<caption><center><u><b><font color="purple">Figure 2:</font></b></u> RNN architecture: many to many</center></caption>
</p>
The character-level language model will be trained on names; which means after we're done with training the model, we'll be able to generate interesting names :).

In this section, we'll go over four main parts:
1. Forward propagation.
2. Backpropagation.
3. Sampling.
4. Fitting the model.

<h3 style="font-family: Georgia; font-size:1.5em;color:purple; font-style:bold">
Forward Propagation</h3>

We'll be using Stochastic Gradient Descent (SGD) where each batch consists of only one example. In other words, the RNN model will learn from each example (name) separately, i.e. run both forward and backward passes on each example and update parameters accordingly. Below are all the steps needed for a forward pass:
- Create a vocabulary dictionary using the unique lower case letters.
    - Create a character to index dictionary that maps each character to its corresponding index in an ascending order. For example, "a" would have index 1 (since python is a zero index language and we'll reserve 0 index to EOS "\n") and "z" would have index 26. We will use this dictionary in converting names into lists of integers where each letter will be represented as one-hot vector.
    - Create an index to character dictionary that maps indices to characters. This dictionary will be used to convert the output of the RNN model into characters which will be translated into names.
- Initialize parameters: weights will be initialized to small random numbers from standard normal distribution to break symmetry and make sure different hidden units learn different things. On the other hand, biases will be initialized to zeros.
    - $W_{hh}$: weight matrix connecting previous hidden state $h^{t - 1}$ to current hidden state $h^t$.
    - $W_{xh}$: weight matrix connecting input $x^t$ to hidden state $h^t$.
    - $b$: hidden state bias vector.
    - $W_{hy}$: weight matrix connecting hidden state $h^t$ to output $o^t$.
    - $c$: output bias vector.
- Convert input $x^t$ and output $y^t$ into one-hot vector each. The dimension of the one-hot vector is vocab_size x 1. Everything will be zero except for the index of the letter at (t) would be 1. In our case, $x^t$ would be the same as $y^t$ shifted to the left where $x^1 = \vec{0}$; however, starting from $t = 2$, $x^{t + 1} = y^{t}$. For example, if we use "imad" as the input, then $y = [3, 4, 1, 2, 0]$ while $x = [\vec{0}, 3, 4, 1, 2]$. Notice that $x^1 = \vec{0}$ and not the index 0. Moreover, we're using "\n" as EOS (end of sentence/name) for each name so that the RNN learns "\n" as any other character so that it knows when to stop generating characters. Therefore, the last target character for all names will be "\n" that represents the end of the name.
- Compute the hidden state using the following formula:
$$h^t = tanh(W_{hh}h^{t-1} + W_{xh}x^t + b)\tag{1}\\{}$$
Notice that we use hyperbolic tangent $(\frac{e^x - e^{-x}}{e^x + e^{-x}})$ as the non-linear function. One of the main advantages of the hyperbolic tangent function is that it resembles the identity function.
- Compute the output layer using the following formula:
$$o^t = W_{hy}h^{t} + c\tag{2}\\{}$$
- Pass the output through softmax layer to normalize the output that allows us to express it as a probability, i.e. all output will be between 0 and 1 and sum up to 1. Below is the softmax formula:
$$y^t = \frac{e^{o^t}}{\sum_ie^{o^t}}\tag{3}\\{}$$
The softmax layer has the same dimension as the output layer which is vocab_size x 1. As a result, $y^t[i]$ is the probability of of index $i$ being the next character at time step (t).
- As mentioned before, the objective of a character-level language model is to minimize the negative log-likelihood of the training sequence. Therefore, the loss function at time (t) and the total loss across all time steps are:
$$\mathcal{L}^t = -\sum_{i = 1}^{T_y}y^tlog\widehat{y^t}\tag{4}\\{}$$
$$\mathcal{L} = \sum_{t = 1}^{T_y}\mathcal{L}^t(\widehat{y^t}, y^t)\tag{5}$$
Since we'll be using SGD, the loss will be noisy and have many oscillations, so it's a good practice to smooth out the loss using exponential weighted average.
- Pass the target character $y^t$ as the next input $x^{t + 1}$ until we finish the sequence.

In [1]:
# Load packages
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

os.chdir("../scripts/")
from character_level_language_model import (initialize_parameters,
                                            initialize_rmsprop,
                                            softmax,
                                            smooth_loss,
                                            update_parameters_with_rmsprop)
os.chdir("../notebooks/")
%matplotlib inline
sns.set_context("notebook")
plt.style.use("fivethirtyeight")

In [2]:
def rnn_forward(x, y, h_prev, parameters):
    """
    Implement one Forward pass on one name.

    Arguments
    ---------
    x : list
        list of integers for the index of the characters in the example
        shifted one character to the right.
    y : list
        list of integers for the index of the characters in the example.
    h_prev : array
        last hidden state from the previous example.
    parameters : python dict
        dictionary containing the parameters.

    Returns
    -------
    loss : float
        cross-entropy loss.
    cache : tuple
        contains three python dictionaries:
            xs -- input of all time steps.
            hs -- hidden state of all time steps.
            probs -- probability distribution of each character at each time
                step.
    """
    # Retrieve parameters
    Wxh, Whh, b = parameters["Wxh"], parameters["Whh"], parameters["b"]
    Why, c = parameters["Why"], parameters["c"]

    # Initialize inputs, hidden state, output, and probabilities dictionaries
    xs, hs, os, probs = {}, {}, {}, {}

    # Initialize x0 to zero vector
    xs[0] = np.zeros((vocab_size, 1))

    # Initialize loss and assigns h_prev to last hidden state in hs
    loss = 0
    hs[-1] = np.copy(h_prev)

    # Forward pass: loop over all characters of the name
    for t in range(len(x)):
        # Convert to one-hot vector
        if t > 0:
            xs[t] = np.zeros((vocab_size, 1))
            xs[t][x[t]] = 1
        # Hidden state
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t - 1]) + b)
        # Logits
        os[t] = np.dot(Why, hs[t]) + c
        # Probs
        probs[t] = softmax(os[t])
        # Loss
        loss -= np.log(probs[t][y[t], 0])

    cache = (xs, hs, probs)
#     print("loss for {} and {} is {}".format(x, y, loss))
    return loss, cache

<h3 style="font-family: Georgia; font-size:1.5em;color:purple; font-style:bold">
Backpropagation</h3>

With RNN based models, the gradient-based technique that will be used is called **Backpropagation Through Time (BPTT)**. We start at last time step $T$ and backpropagate loss function w.r.t. all parameters across all time steps and sum them up (see figure 3).
<p align="left">
<img src="posts_images/char_level_model/backprop.PNG"; style="width: 800px; height: 400px"><br>
<caption><center><u><b><font color="purple">Figure 3:</font></b></u> Backpropagation Through Time (BPTT)</center></caption>
</p>
In addition, since recurrent networks are known to have steep cliffs (sudden steep decrease in $\mathcal{L}$), gradients may overshoot the minimum and undo a lot of the work that was done even if we are using adaptive learning methods such as RMSProp. The reason for that is that because gradient is a linear approximation of the loss function and may not capture information further than the point it was evaluated on such as the curvature of loss curve. Therefore, it's a common practice to clip the gradients to be in the interval [-maxValue, maxValue]. For this exercise, we'll clip the gradients to be in the interval [-5, 5]. That means if the gradient is > 5 or < -5, it would be clipped to 5 and -5 respectively. Below are all the formulas needed to compute the gradients w.r.t. all parameters at all time steps.

$$\nabla_{o^t}\mathcal{L} = \widehat{y^t} - y^t\tag{6}\\{}$$
$$\nabla_{W_{hy}}\mathcal{L} = \sum_t \nabla_{o^t}\mathcal{L}\cdot{h^t}^T\tag{7}\\{}$$
$$\nabla_{c}\mathcal{L} = \sum_t \nabla_{o^t}\mathcal{L} \tag{8}\\{}$$
$$\nabla_{h^t}\mathcal{L} = W_{hy}^T\cdot\nabla_{o^t}\mathcal{L} + \underbrace { W_{hh}^T\cdot\nabla_{h^{t + 1}}\mathcal{L} * (1 - tanh(W_{hh}h^{t} + W_{xh}x^{t + 1} + b) ^ 2)}_{dh_{next}} \tag{9}\\{}$$
$$\nabla_{h^{t - 1}}\mathcal{L} = W_{hh}^T\cdot\nabla_{h^t}\mathcal{L} * (1 - tanh(h^t) ^ 2)\tag{10}\\{}$$
$$\nabla_{x^t}\mathcal{L} = W_{xh}^T\cdot\nabla_{h^t}\mathcal{L} * (1 - tanh(W_{hh}\cdot h^{t-1} + W_{xh}\cdot x^t + b) ^ 2)\tag{11}\\{}$$
$$\nabla_{W_{hh}}\mathcal{L} = \sum_t \nabla_{h^t}\mathcal{L} * (1 - tanh(W_{hh}\cdot h^{t-1} + W_{xh}\cdot x^t + b) ^ 2)\cdot{h^{t - 1}}^T\tag{12}\\{}$$
$$\nabla_{W_{xh}}\mathcal{L} = \sum_t \nabla_{h^t}\mathcal{L} * (1 - tanh(W_{hh}\cdot h^{t-1} + W_{xh}\cdot x^t + b) ^ 2) . {x^t}^T\tag{13}\\{}$$
$$\nabla_{b}\mathcal{L} = \sum_t \nabla_{h^t}\mathcal{L} * (1 - tanh(h^t) ^ 2) \tag{14}\\{}$$

Note that at last time step $T$, we'll initialize $dh_{next}$  to zeros since we can't get values from future. To stabilize the update at each time step since SGD may have so many oscillations, we'll be using one of the adaptive learning methods' optimizer. More specifically, [**Root Mean Squared Propagation (RMSProp)**](https://nbviewer.jupyter.org/github/ImadDabbura/Deep-Learning/blob/master/notebooks/Optimization-Algorithms.ipynb) which tends to have acceptable performance.

In [3]:
def clip_gradients(gradients, max_value):
    """
    Implements gradient clipping element-wise on gradients to be between the
    interval [-max_value, max_value].

    Arguments
    ----------
    gradients : python dict
        dictionary that stores all the gradients.
    max_value : scalar
        edge of the interval [-max_value, max_value].

    Returns
    -------
    gradients : python dict
        dictionary where all gradients were clipped.
    """
    for grad in gradients.keys():
        np.clip(gradients[grad], -max_value, max_value, out=gradients[grad])

    return gradients


def rnn_backward(y, parameters, cache):
    """
    Implements Backpropagation on one name.

    Arguments
    ---------
    y : list
        list of integers for the index of the characters in the example.
    parameters : python dict
        dictionary containing the parameters.
    cache : tuple
            contains three python dictionaries:
                xs -- input of all time steps.
                hs -- hidden state of all time steps.
                probs -- probability distribution of each character at each time
                    step.

    Returns
    -------
    grads : python dict
        dictionary containing all the gradients.
    h_prev : array
        last hidden state from the current example.
    """
    # Retrieve xs, hs, and probs
    xs, hs, probs = cache

    # Initialize all gradients to zero
    dh_next = np.zeros_like(hs[0])

    parameters_names = ["Whh", "Wxh", "b", "Why", "c"]
    grads = {}
    for param_name in parameters_names:
        grads["d" + param_name] = np.zeros_like(parameters[param_name])

    # Iterate over all time steps in reverse order starting from Tx
    for t in reversed(range(len(xs))):
        dy = np.copy(probs[t])
        dy[y[t]] -= 1
        grads["dWhy"] += np.dot(dy, hs[t].T)
        grads["dc"] += dy
        dh = np.dot(parameters["Why"].T, dy) + dh_next
        dhraw = (1 - hs[t] ** 2) * dh
        grads["dWhh"] += np.dot(dhraw, hs[t - 1].T)
        grads["dWxh"] += np.dot(dhraw, xs[t].T)
        grads["db"] += dhraw
        dh_next = np.dot(parameters["Whh"].T, dhraw)
        # Clip the gradients using [-5, 5] as the interval
        grads = clip_gradients(grads, 5)
    # Get the last hidden state
    h_prev = hs[len(xs) - 1]

    return grads, h_prev

In [4]:
print("hello")

hello


<h3 style="font-family: Georgia; font-size:1.5em;color:purple; font-style:bold">
Sampling</h3><br>
Sampling is what makes the text generated by the RNN at each time step an interesting/creative text. On each time step (t), the RNN output the conditional probability distribution of the next character given all the previous characters, i.e. $P(c_t/c_1, c_2, ..., c_{t-1})$. Let's assume that we are at time step $t = 3$ and we're trying to predict the third character, the conditional probability distribution is: $P(c_3/c_1, c_2) = (0.2, 0.3, 0.4, 0.1)$. We'll have two extremes:
1. Maximum entropy: the character will be picked randomly using uniform probability distribution; which means that all characters in the vocabulary dictionary are equally likely. Therefore, we'll end up with maximum randomness in picking the next character and the generated text will not be either meaningful or sound real.
2. Minimum entropy: the character with the highest conditional probability will be picked on each time step. That means next character will be what the model estimates to be the right one based on the training text and learned parameters. As a result, the name generated will be both meaningful and sound real. However, it will also be repetitive and interesting since all the parameters were optimized to learn joint probability distribution in predicting the next character.

As we increase randomness, text will loose local structure; however, as we decrease randomness, the generated text will sound more real and start to preserve its local structure. For this exercise, we will sample from the distribution that's generated by the model which can be seen as an intermediate level of randomness between maximum and minimum entropy (see figure 4). Using this sampling strategy on the above distribution, the index 0 has $20$% probability of being picked, while index 2 has $40$% probability to be picked.
<p align="left">
<img src="posts_images/char_level_model/sampling.PNG"; style="width: 800px; height: 400px"><br>
<caption><center><u><b><font color="purple">Figure 4:</font></b></u> Sampling: An example of predicting next character using character-level language model</center></caption>
</p>
Therefore, sampling will be used at test time to generate names character by character.

In [5]:
def sample(parameters, idx_to_chars, chars_to_idx, n):
    """
    Implements sampling of a squence of n characters characters length. The
    sampling will be based on the probability distribution output of RNN.

    Arguments
    ---------
    parameters : python dict
        dictionary storing all the parameters of the model.
    idx_to_chars : python dict
        dictionary mapping indices to characters.
    chars_to_idx : python dict
        dictionary mapping characters to indices.
    n : scalar
        number of characters to output.

    Returns
    -------
    sequence : str
        sequence of characters sampled.
    """
    # Retrienve parameters, shapes, and vocab size
    Whh, Wxh, b = parameters["Whh"], parameters["Wxh"], parameters["b"]
    Why, c = parameters["Why"], parameters["c"]
    n_h, n_x = Wxh.shape
    vocab_size = c.shape[0]

    # Initialize a0 and x1 to zero vectors
    h_prev = np.zeros((n_h, 1))
    x = np.zeros((n_x, 1))

    # Initialize empty sequence
    indices = []
    idx = -1
    counter = 0
    while (counter <= n and idx != chars_to_idx["\n"]):
        # Fwd propagation
        h = np.tanh(np.dot(Whh, h_prev) + np.dot(Wxh, x) + b)
        o = np.dot(Why, h) + c
        probs = softmax(o)

        # Sample the index of the character using generated probs distribution
        idx = np.random.choice(vocab_size, p=probs.ravel())

        # Get the character of the sampled index
        char = idx_to_chars[idx]

        # Add the char to the sequence
        indices.append(idx)

        # Update a_prev and x
        h_prev = np.copy(h)
        x = np.zeros((n_x, 1))
        x[idx] = 1

        counter += 1
    sequence = "".join([idx_to_chars[idx] for idx in indices if idx != 0])

    return sequence

<h3 style="font-family: Georgia; font-size:1.5em;color:purple; font-style:bold">
Fitting the model</h3><br>
After covering all the concepts/intuitions behind character-level language model, now we're ready to fit the model. We'll use the default settings for RMSProp's hyperparameters and run the model for 100 iterations. On each iteration, we'll print out one sampled name and smoothed loss to see how the names generated start to get more interesting with more iterations as well as the loss will start decreasing. When done with fitting the model, we'll plot the loss function and generate some names.

In [ ]:
def model(
        file_path, chars_to_idx, idx_to_chars, hidden_layer_size, vocab_size,
        num_epochs=10, learning_rate=0.01):
    """
    Implements RNN to generate characters.

    Arguments
    ---------
    file_path : str
        path to the file of the raw data.
    num_epochs : int
        number of passes the optimization algorithm to go over the training
        data.
    learning_rate : float
        step size of learning.
    chars_to_idx : python dict
        dictionary mapping characters to indices.
    idx_to_chars : python dict
        dictionary mapping indices to characters.
    hidden_layer_size : int
        number of hidden units in the hidden layer.
    vocab_size : int
        size of vocabulary dictionary.

    Returns
    -------
    parameters : python dict
        dictionary storing all the parameters of the model.
    overall_loss : list
        list stores smoothed loss per epoch.
    """
    # Get the data
    with open(file_path) as f:
        data = f.readlines()
    examples = [x.lower().strip() for x in data]
    # examples is a line (which is just a name)
    # let us computer the loss PER name
    
    # Initialize parameters
    parameters = initialize_parameters(vocab_size, hidden_layer_size, load=False)

    # Initialize Adam parameters
    s = initialize_rmsprop(parameters)

    # Initialize loss
    #     actually the loss can be saved as well, but this is OK too
    smoothed_loss = -np.log(1 / vocab_size) * 7  
    

    # Initialize hidden state h0 and overall loss
    h_prev = np.zeros((hidden_layer_size, 1))
    overall_loss = []

    # Iterate over number of epochs
    for epoch in range(num_epochs):
        print(f"\033[1m\033[94mEpoch {epoch}")
        print(f"\033[1m\033[92m=======")

        # Sample one name
        print(f"""Sampled name: {sample(parameters, idx_to_chars, chars_to_idx,
            10).capitalize()}""")
        print(f"Smoothed loss: {smoothed_loss:.4f}\n")

        # Shuffle examples
        np.random.shuffle(examples)

        # Iterate over all examples (SGD)
        for example in examples:
            
            # run thru the example and compute the loss
            
            x = [None] + [chars_to_idx[char] for char in example]
            y = x[1:] + [chars_to_idx["\n"]]
            # Fwd pass
            loss, cache = rnn_forward(x, y, h_prev, parameters)
            
#             print the loss per example
            print("loss for {} is {}".format(example, loss))
            
            # Compute smooth loss
            smoothed_loss = smooth_loss(smoothed_loss, loss)
            # Bwd pass
            grads, h_prev = rnn_backward(y, parameters, cache)
            # Update parameters
            parameters, s = update_parameters_with_rmsprop(
                parameters, grads, s)
            
            # ideally, we can save the parameters here; 
            # s => save the parameters  
        overall_loss.append(smoothed_loss)
#     save_model(parameters)
    
    return parameters, overall_loss

import pickle

#  now that we can save the parameters, we can train it on a new corpus...
# test set: training losses! but we need to see how the loss is 

def save_model(parameters):
    print("hey we going to save!")
    with open("sample_file.pk1", "wb") as output:
        pickle.dump(parameters, output, pickle.HIGHEST_PROTOCOL)
    
    
    

In [ ]:
# Load names
# this is where we get the data from (think of it like a dictionary)
data = open("../data/names_augmented_with_nums.txt", "r").read()

# Convert characters to lower case
data = data.lower()

# Construct vocabulary using unique characters, sort it in ascending order,
# then construct two dictionaries that maps character to index and index to
# characters.
chars = list(sorted(set(data)))
chars_to_idx = {ch:i for i, ch in enumerate(chars)}
idx_to_chars = {i:ch for ch, i in chars_to_idx.items()}

# Get the size of the data and vocab size
data_size = len(data)
vocab_size = len(chars_to_idx)
print(f"There are {data_size} characters and {vocab_size} unique characters.")

# Fitting the model
# this is the actual data we want to run the model on
parameters, loss = model("../data/names_augmented_with_nums.txt", chars_to_idx, idx_to_chars, 100, vocab_size, 100, 0.01)

save_model(parameters)

# Plotting the loss
plt.plot(range(len(loss)), loss)
plt.xlabel("Epochs")
plt.ylabel("Smoothed loss");

There are 36165 characters and 51 unique characters.
Epoch 0
Sampled name: !)]upy
Smoothed loss: 27.5228

loss for ashley is 27.522480049216455
loss for xiomara is 31.442464745303063
loss for marilee is 31.37267874502054
loss for mireya is 27.410993279216363
loss for bart is 19.573322271713643
loss for louise is 27.362176548765905
loss for jamaal is 27.0712874421518
loss for irmgard is 30.35621921614677
loss for tawny is 21.879420221091372
loss for tereasa is 22.684396227624198
loss for barbie is 20.922574911934937
loss for leatha is 19.406064594331635
loss for kimbra is 21.784608076790118
loss for brandon is 28.002474850043367
loss for alyce is 18.57442530808069
loss for sherlene is 26.762546889083257
loss for lelah is 15.62411334478367
loss for hassan is 20.122052133892485
loss for debbie is 21.715672706650633
loss for robbi is 18.657122283483794
loss for dalila is 18.066114300401953
loss for carman is 20.59540506714307
loss for stacie is 20.528937124245758
loss for windy is 21.12418

loss for stephany is 28.617129741017724
loss for tyrell is 18.967132019988203
loss for eugenie is 22.331075182859887
loss for latrisha is 25.31859962318663
loss for verlie is 19.34644837831439
loss for kira is 13.98921962370014
loss for diedre is 18.720902206771175
loss for zora is 16.70111330545131
loss for wilbur is 24.951641023168957
loss for eveline is 21.23618893014574
loss for leonila is 19.955397189307487
loss for gennie is 17.94174938422101
loss for kendall is 22.22525848752912
loss for nakita is 19.29645560818457
loss for celinda is 21.105929972943994
loss for lettie is 17.15171641048838
loss for ivan is 14.27082504040833
loss for charline is 24.133818074092066
loss for kasie is 16.451641292844233
loss for ardella is 19.9762432330236
loss for jacquelin is 35.21185148875629
loss for adolph is 23.11767227928632
loss for alycia is 18.71844811432052
loss for freeman is 23.508245508659098
loss for kraig is 18.021378332202172
loss for chaya is 17.14835047681869
loss for lucio is 17.

loss for renae is 14.039320665744235
loss for leanne is 16.911121563068285
loss for madonna is 21.162433012738774
loss for hal is 10.679461222249016
loss for jina is 13.015281165854786
loss for saturnina is 26.388705385194868
loss for yee is 10.163576096133756
loss for ailene is 15.889510261599806
loss for matt is 14.06054725005317
loss for moses is 18.039956700194807
loss for nicolle is 21.973324092722645
loss for jenna is 14.236075616892103
loss for ismael is 19.023618498927554
loss for bobby is 23.895066911872306
loss for eilene is 16.003398880587206
loss for ana is 8.060838369137775
loss for alleen is 16.528862992207
loss for tinisha is 21.19874060996419
loss for lourdes is 24.734934774998216
loss for anne is 9.99327410557002
loss for coral is 16.59643425001083
loss for mavis is 18.542164606047333
loss for lashell is 20.860814878033317
loss for dann is 12.599273929082052
loss for sabra is 15.770810980237215
loss for albertha is 24.700311649953825
loss for cher is 14.596381044069107

loss for rosina is 16.50954726409183
loss for amada is 15.596713588928441
loss for russell is 24.84123474438306
loss for katlyn is 20.68579869040048
loss for vikki is 19.469834937476087
loss for cyril is 17.12676940126351
loss for maegan is 19.74315547556727
loss for betsey is 20.4108293725782
loss for jeniffer is 28.1378972115818
loss for maria is 13.224251900878883
loss for frederick is 29.4430000317729
loss for fannie is 18.370102864944133
loss for ashlie is 18.10724103095225
loss for cristine is 22.693212485892673
loss for jeanine is 19.277947375045397
loss for marry is 15.228716236948113
loss for miguelina is 27.18750679598636
loss for lakia is 13.930207423651352
loss for suk is 13.40955557864272
loss for marlen is 16.4617688133422
loss for latoyia is 21.31276054821725
loss for kyra is 12.851467388207158
loss for magaly is 19.226627725908532
loss for vicente is 21.39523353823758
loss for reginald is 25.82606070046724
loss for josef is 19.83304122694788
loss for deetta is 18.470796

loss for cody is 14.734666530429621
loss for marianna is 17.952831408592946
loss for jacquelyne is 33.442863709162225
loss for chauncey is 26.984139353276124
loss for dorethea is 22.232418073497325
loss for ardelia is 18.401628621974826
loss for leigha is 17.35063437307977
loss for connie is 17.30168170331872
loss for altagracia is 26.801131123656187
loss for lawerence is 27.286561215834073
loss for thaddeus is 27.03147474979879
loss for hettie is 15.71520296988781
loss for danny is 14.541369960186165
loss for antonio is 19.869194921834545
loss for shanell is 20.83761486491873
loss for sanjuana is 21.723655737060266
loss for pasty is 16.836128938373342
loss for lauralee is 22.230365280539647
loss for chelsea is 18.83150014656689
loss for blake is 18.45271143587119
loss for betsy is 17.989483903818986
loss for anneliese is 22.609911995060955
loss for lanie is 11.087603063435107
loss for aida is 11.217734244750634
loss for patti is 17.399658988289417
loss for kaci is 15.436178218770674
l

...

loss for herb is 14.502451935069388
loss for carleen is 18.177082104961105
loss for constance is 21.59187745964509
loss for gerardo is 18.748146324325305
loss for joshua is 17.921091717444106
loss for wilton is 17.692928941677398
loss for jordon is 14.252567151449107
loss for adrian is 15.193860106408005
loss for kathrine is 19.41283416841451
loss for jarvis is 15.385517565610193
loss for rebekah is 27.00090283747823
loss for loan is 12.596962203336421
loss for lewis is 16.71576053555119
loss for michaela is 20.76839473121634
loss for madalyn is 17.94893652737935
loss for monica is 13.352042826047212
loss for karmen is 13.549179584850291
loss for becky is 16.660118506395587
loss for roberta is 17.347434453562283
loss for lyda is 11.341295137184108
loss for tim is 11.740814743681836
loss for eileen is 17.539662190365707
loss for signe is 16.43918724184209
loss for aliza is 16.83398659137661
loss for lamont is 17.150788519752055
loss for mack is 15.89418775210201
loss for delilah is 21.7

loss for alexandria is 31.090484636797203
loss for keri is 11.508730939494821
loss for elizabet is 27.50864754658491
loss for valerie is 16.42447356342761
loss for mabel is 11.890129803990162
loss for noelia is 15.858116378158169
loss for danette is 14.554078082055309
loss for catherin is 19.591241942030006
loss for lashonda is 17.692913893508845
loss for dewitt is 18.069695088337742
loss for tess is 14.532755566987015
loss for glynis is 17.995945057899892
loss for lino is 11.207923829532575
loss for pia is 11.318040247554006
loss for gayle is 14.391909863102693
loss for salina is 11.492597419307808
loss for gertrud is 25.132913736727662
loss for ruby is 15.175917591146892
loss for veronica is 18.451805655637532
loss for elouise is 19.914432104520106
loss for eulah is 16.522366553187506
loss for kimi is 11.935569924748112
loss for fernanda is 18.861342245831285
loss for cinderella is 25.300592172142117
loss for darryl is 16.472062520022405
loss for annis is 12.727848846196816
loss for 

loss for tami is 11.633992842206549
loss for giovanna is 22.733206416382217
loss for geraldine is 21.548209542367722
loss for simone is 16.230773923268757
loss for elois is 16.299712034746072
loss for georgine is 22.540412124827675
loss for aleta is 13.003759030150945
loss for antoine is 20.52789649241336
loss for sheridan is 15.824548306771735
loss for elicia is 13.849056789958805
loss for dedra is 13.466250943455325
loss for janna is 9.99001611276086
loss for lorita is 13.333713961264513
loss for nick is 16.365498336757227
loss for deloras is 19.498399495352118
loss for jerrell is 17.26904236768217
loss for korey is 14.765635717257414
loss for deja is 12.426460463950109
loss for larae is 13.828372117313258
loss for junie is 13.449428130717699
loss for kasey is 14.285129495911868
loss for brice is 14.175973777947076
loss for fran is 11.67835824798126
loss for shakita is 17.986016849885864
loss for era is 8.471265694181362
loss for therese is 16.973616293024406
loss for paz is 16.01447

loss for mana is 9.420818208061284
loss for chang is 14.59475210578606
loss for vanessa is 18.35039467819083
loss for mollie is 12.970807201819323
loss for jeanett is 16.639392246310443
loss for gus is 14.698043368280638
loss for tam is 12.479555995156966
loss for patrica is 19.66671161090799
loss for misti is 10.173783221814531
loss for dotty is 14.682383894799258
loss for lottie is 14.569939993865155
loss for kanisha is 15.277998186466627
loss for wendolyn is 22.632770349271887
loss for timmy is 18.049578601629023
loss for ladonna is 18.00219528936142
loss for bradly is 17.55854124218699
loss for jamey is 13.985083203029598
loss for carmel is 14.296060291242279
loss for dario is 13.25324603470852
loss for zulma is 15.970463477325453
loss for laurene is 18.00791941139305
loss for rosanne is 15.228505260592827
loss for shaunte is 17.182948385218623
loss for tamela is 12.6636096541431
loss for keiko is 17.939303839212368
loss for hayley is 16.015668007992602
loss for shani is 11.5425527

loss for lowell is 17.420879540692162
loss for donovan is 20.819064058375
loss for lauretta is 17.95169014087953
loss for genna is 11.091793190697842
loss for prudence is 25.546380296443985
loss for antione is 19.794471253962264
loss for zetta is 14.660715533571855
loss for heike is 14.894588703205129
loss for vanna is 10.085009875235443
loss for violette is 20.781434759495717
loss for tashina is 14.352176662442368
loss for mao is 12.204033282774898
loss for cecila is 16.34298088674771
loss for tawana is 15.325020722505332
loss for shantay is 15.571809248144346
loss for irena is 12.504458145792867
loss for sherita is 12.359331203305677
loss for noma is 11.504994364519579
loss for dominga is 18.284752577769193
loss for vicenta is 18.809052174322957
loss for gaston is 15.1921346282755
loss for wilda is 14.734884621347716
loss for dacia is 12.158941609000044
loss for johanna is 15.989769469538741
loss for sharan is 11.209024266363445
loss for natalie is 17.290226934961172
loss for nita is

As you may notice, the names generated started to get more interesting after 15 epochs. One of the interesting names is "Yasira" which is an Arabic name :).

<h2 style="font-family: Georgia; font-size:2em;color:purple; font-style:bold">
Conclusion</h2><br>
Statistical language models are very crucial in Natural Language Processing (NLP) such as speech recognition and machine translation. We demonstrated in this notebook the main concepts behind statistical language models using character-level language model. The task of this model is generate names character by character using names obtained from census data that were consisted of 5,163 names. Below are the main key takeaways:
- If we have more data, bigger model, and train longer we may get more interesting results. However, to get a very interesting results, we should instead use **Long Short_Term Memory (LSTM)** model with more than one layer deep. People have used 3 layers deep LSTM model with dropout and were able to generate very interesting results when applied on cook books and Shakespeare poems. LSTM models outperform simple RNN due to its ability in capturing longer time dependencies.
- With the sampling technique we're using, don't expect the RNN to generate meaningful sequence of characters (names).
- We used in this notebook each name as its own sequence; however, we may be able to speed up learning and get better results if we increase the batch size lets say from one name to a sequence of 50 characters.
- We can control the level of randomness using the sampling strategy. Here, we balanced between what the model thinks its the right character and the level of randomness.